# 🚀 VTrackAI Studio – Full Colab GPU (SAM3 + Frontend)

**Backend + Frontend רצים בתוך Colab על T4 GPU.**

**Flow:**
1. Runtime → **T4 GPU**
2. Sidebar → **Secrets** → Add `HF_TOKEN`
3. Runtime → **Restart session**
4. Ctrl+F9 (**Run all**) – ובסוף תקבל קישורים לפתיחת ה‑UI וה‑API בדפדפן.

**Important:** אין שימוש ב‑ngrok / cloudflare / localtunnel. Colab ייתן לך URLs ל‑ports.

## 1. GPU Check

In [ ]:
import torch
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ Set runtime type to GPU (T4) and rerun.")

## 2. Install Backend + Frontend Dependencies

In [ ]:
!rm -rf /content/sam3 /content/vtrack-ai-studio

# PyTorch + core libs (GPU)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart opencv-python librosa huggingface_hub nest-asyncio

# Node / frontend deps
!apt-get -y install nodejs npm > /dev/null 2>&1

print("✅ Dependencies installed")

## 3. Clone SAM3 + VTrackAI and Install

In [ ]:
%cd /content
!git clone https://github.com/facebookresearch/sam3
!pip install -e ./sam3 --quiet

!git clone https://github.com/Amitro123/vtrack-ai-studio
%cd /content/vtrack-ai-studio/backend
!pip install -r requirements.txt --quiet

print("✅ SAM3 + VTrackAI backend ready")

## 4. HF Login from Secrets + Download SAM3 Checkpoint

In [ ]:
from google.colab import userdata
from huggingface_hub import login, hf_hub_download
import os

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)
print('✅ HF logged in from Secrets')

os.makedirs("/content/vtrack-ai-studio/backend/checkpoints/sam3", exist_ok=True)
%cd /content/vtrack-ai-studio/backend

checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir="./checkpoints/sam3",
    token=HF_TOKEN
)
print(f"✅ SAM3 checkpoint: {checkpoint_path}")
print(f"📁 Size: {os.path.getsize(checkpoint_path)/1e9:.1f} GB")

sam3_pt = "./checkpoints/sam3/sam3.pt"
try:
    os.symlink(sam3_pt, "./checkpoints/sam3/sam3_hiera_large.pt")
except FileExistsError:
    pass
os.makedirs("./checkpoints/sam3/checkpoints", exist_ok=True)
try:
    os.symlink(sam3_pt, "./checkpoints/sam3/checkpoints/sam3_hiera_tiny.pt")
except FileExistsError:
    pass
print("✅ Symlinks ready for backend")

## 5. Start FastAPI Backend on GPU (Port 8000)

In [ ]:
import os, time, multiprocessing as mp, requests, nest_asyncio
import uvicorn

os.chdir('/content/vtrack-ai-studio/backend')
print(f"✅ Backend dir: {os.getcwd()}")

!pkill -f uvicorn || true
time.sleep(2)
nest_asyncio.apply()

def run_uvicorn():
    uvicorn.run("server:app", host="0.0.0.0", port=8000, log_level="info")

p = mp.Process(target=run_uvicorn)
p.start()
print(f"🚀 Uvicorn started with PID: {p.pid}")
time.sleep(8)

try:
    r = requests.get("http://127.0.0.1:8000/api/health", timeout=10)
    print("✅ Health:", r.json())
except Exception as e:
    print("❌ Health failed:", e)

## 6. Configure Frontend to Talk to Local Backend

In [ ]:
%cd /content/vtrack-ai-studio
with open('.env', 'w') as f:
    f.write('VITE_API_URL=http://localhost:8000\n')
print('✅ .env updated with VITE_API_URL=http://localhost:8000')

## 7. Start Frontend (Vite) – Port 4173

הרץ תא זה **בסוף** ותשאיר אותו פתוח. Colab יציג קישור לפתיחת ה‑UI בדפדפן.

In [ ]:
%cd /content/vtrack-ai-studio
!npm install --silent

print('\n🚀 Starting Vite dev server on port 4173 (host=0.0.0.0)...')
print('ℹ️ Colab usually shows a link like: https://xxxx-4173.colab.googleusercontent.com')
print('   Open that URL in your browser to use VTrackAI Studio UI.\n')

!npm run dev -- --host 0.0.0.0 --port 4173